In [51]:
# 导入需要的包
import random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import base64
import  requests
import re
import pandas as pd

In [52]:
HTTP_HEADER ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
COOKIES = {}
# 从cooki中读取键值对的json串,eval为json对象
with open('cookie.ini','r') as f:
    cookie = ''
    lines = f.readlines()
    for line in lines:
        cookie += line
    COOKIES = eval(cookie)

def get_text_post(url,data,cookies = COOKIES):
    html = requests.post(url=url,data = data,headers=HTTP_HEADER,cookies=cookies)
    return html.text



In [53]:
# 加密专用
class Encrypter:

    def __init__(self):
        # 加密思路：
        # 1、使用ase_iv,ase_mode,ase_key,对需要提交的字符串进行第一次加密;
        # 2、随机生成一个16位的秘钥，结合aes算法，对数据进行第二次加密;  加密后得到 => encText
        # 3、将随机生成的秘钥，采用非对称加密的方式，使用公钥加密；加密后得到 => encKey
        # 4、将2、3得到的数据作为form表单的参数提交至服务器
        self.sequence = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
        self.aes_iv = '0102030405060708'
        self.aes_mode = AES.MODE_CBC
        self.aes_key = '0CoJUm6Qyw8W8jud'
        self.rsa_public_key = '010001'
        self.rsa_modulus = '00e0b509f6259df8642dbc35662901'\
                    '477df22677ec152b5ff68ace615bb7b7'\
                    '25152b3ab17a876aea8a5aa76d2e417629'\
                    'ec4ee341f56135fccf695280104e0312e'\
                    'cbda92557c93870114af6c9d05c4f7f0c'\
                    '3685b7a46bee255932575cce10b424d813c'\
                    'fe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7'

    def random_ase_key(self):
        random_key = random.sample(self.sequence,16)
        random_key = ''.join(random_key)
        return random_key
    
    def aes_encrypt(self,plain_text,mode=AES.MODE_CBC,iv=None,key=None):
        # data padding
        # data_to_padd : need a byte objectencode return a bytes object,
        # The block boundary to use for padding
        plain_text = pad(data_to_pad=plain_text.encode(),block_size= AES.block_size)
        # create a encrypt object
        aes = AES.new(key=key.encode(),mode=mode,iv=iv.encode())
        # get the encrypted data
        encrypted_data = aes.encrypt(plaintext=plain_text)
        # use base64 to transfer the encrypted data
        # why ? 参考：
        result = base64.b64encode(encrypted_data).decode()
        return result

    def rsa_encrypt(self,data, public_key, modulus):
        # 部分情况下的运算会导致出现位数不足256位，需要进行填充
        num = pow(int(data[::-1].encode().hex(), 16), int(public_key, 16), int(modulus, 16))
        result = format(num, 'x').zfill(256)
        return result
    def aes_decrypt(self,encode_text,mode=AES.MODE_CBC,iv=None,key=None):
        # use base64 decode the data
        encode_data = base64.b64decode(encode_text.encode())
        # 
        aes = AES.new(key=key.encode(),mode=mode,iv=iv.encode())
        decrypted_data = aes.decrypt(encode_data)
        decrypted_data = unpad(decrypted_data,AES.block_size)
        return decrypted_data.decode()
    
    def get_encrypt(self,plain_text):
        random_key = self.random_ase_key()
        encode_data = self.aes_encrypt(plain_text=plain_text,mode=self.aes_mode,iv=self.aes_iv,key=self.aes_key)
        # 加密后再使用random_key加密
        encText = self.aes_encrypt(plain_text=encode_data,mode=self.aes_mode,iv=self.aes_iv,key=random_key)
        # 使用公钥对 random_key 进行加密，加密后传给服务器
        encKey = self.rsa_encrypt(random_key,self.rsa_public_key,self.rsa_modulus)

        form_data = {
            'encSecKey':encKey, #为何有时候是255位？计算问题...自动填充0即可
            'params':encText
            }
        return form_data

In [54]:
class MusicDownloader:
    def __init__(self) :
        self.url = 'https://music.163.com/weapi/cloudsearch/get/web?csrf_token='
        self.download_url = 'https://music.163.com/weapi/song/enhance/player/url/v1?csrf_token='
        self.comment_url = 'https://music.163.com/weapi/comment/resource/comments/get?csrf_token='
        self.vip_download_url = 'https://music.163.com/weapi/song/enhance/player/url/v1?csrf_token={}'.format(COOKIES['__csrf'])

    def get_dic(self,search_data):
        text = get_text_post(self.url,search_data)
        text = re.sub(r'null|false|true','None',text)
        text = eval(text)
        result_dic = {}
        #print(text)
        #print(text.keys())
        for song in text['result']['songs']:
            result_dic[song['id']] = song['name']
        return result_dic
    
    def download_music(self,music,music_name):
        text = get_text_post(self.download_url,music)
        text = re.sub(r'null|false|true','None',text)
        text = eval(text)
        url = text['data'][0]['url']
        if url==None:
            print('该歌曲需要vip用户或者单曲购买后才能下载，名称:',music_name)
            return None
        print('开始下载歌曲:',music_name)
        print(url)
        data = requests.get(url)
        with open('download/'+music_name+'.m4a','wb') as f:
            f.write(data.content)
        return text
    
    def download_vip_music(self,music,music_name):
        print(self.vip_download_url)
        text = get_text_post(self.vip_download_url,music)
        text = re.sub(r'null|false|true','None',text)
        text = eval(text)
        url = text['data'][0]['url']

        print('开始下载歌曲---',music_name)
        print(url)
        data = requests.get(url)
        with open('download/'+music_name+'.m4a','wb') as f:
            f.write(data.content)
        return text
        
    def download_hot_comment(self,music,music_name):
        text = get_text_post(self.comment_url,music)
        text = re.sub(r'null|false|true','None',text)
        text = eval(text)
        music_hot_comments = []
        print('开始下载---{}---的评论'.format(music_name))
        if text['data']['hotComments'] == None:
            return None
        for comment in text['data']['hotComments']:
            user = comment['user']['nickname']
            content =  comment['content']
            likedCount = comment['likedCount']
            music_hot_comments.append([user,content,likedCount])
        user = []
        content = []
        likedCount = []
        singer = []
        for i in range(len(music_hot_comments)):
            singer.append(music_name)
            user.append(music_hot_comments[i][0].encode('utf-8','replace').decode('utf-8'))
            content.append(music_hot_comments[i][1].encode('utf-8','replace').decode('utf-8'))
            likedCount.append(music_hot_comments[i][2])
        data = pd.DataFrame(
            {'singer':singer 
            ,'comment_user':user,
            'content':content,
            'likedCount':likedCount}
        )
        # print(data)
        data.to_csv('download/'+music_name+'.csv',encoding='utf_8_sig')
        return music_hot_comments

    

In [55]:
def start(s_):

    search_data_paras = '{{"hlpretag":"<span class=\\"s-fc7\\">","hlposttag":"</span>"'\
                ',"s":"{}","type":"1",'\
                '"offset":"0","total":"true","limit":"30","csrf_token":""}}'
    music_down_sequence_paras = '{{"ids":"[{}]","level":"standard","encodeType":"aac","csrf_token":""}}'
    music_comments_sequence_paras = '{{"rid":"R_SO_4_{0}",'\
                            '"threadId":"R_SO_4_{1}","pageNo":"1","pageSize":"20"'\
                            ',"cursor":"-1","offset":"0","orderType":"1","csrf_token":""}}'

    encrpter = Encrypter()
    music_downloader = MusicDownloader()
    
    search_data = search_data_paras.format(s_)
    form_data = encrpter.get_encrypt(search_data)
    music_dic = music_downloader.get_dic(form_data)

    # print(music_dic)
    ## 使用pandas存储表格,便于后续使用
    song_ids = []
    song_names = []
    for songid,songname in music_dic.items():
        song_ids.append(songid)
        song_names.append(songname)
    pd_music = pd.DataFrame(
        {'song_id':song_ids,'song_names':song_names}
    )
    print('下面将下载以下歌曲.......')
    print(pd_music)
    for key,item in music_dic.items():
        
        song_id = key

        music_down_sequence=music_down_sequence_paras.format(song_id)
        form_data = encrpter.get_encrypt(music_down_sequence)
        download_result = music_downloader.download_music(form_data,music_dic[song_id])
        if download_result ==None:
            continue

        music_comments_sequence = music_comments_sequence_paras.format(song_id,song_id)
        form_data = encrpter.get_encrypt(music_comments_sequence)
        text = music_downloader.download_hot_comment(form_data,music_dic[song_id])
        if text:
            continue
            # print(text)
        else:
            print('没有热评')

In [56]:
s_ = input('a singer or song:')
start(s_)

下面将下载以下歌曲.......
       song_id song_names
0       346089       海阔天空
1   1855073420       光辉岁月
2   1857483755       真的爱你
3       347597       不再犹豫
4   1855074234       灰色轨迹
5       346085      谁伴我闯荡
6       347230       海阔天空
7   1857483489        冷雨夜
8       347758         大地
9       347241         情人
10  1857483487        喜欢你
11  1855074232      Amani
12      346591       无尽空虚
13      347712      无悔这一生
14    32701152       灰色轨迹
15      346079       午夜怨曲
16      346576       光辉岁月
17      346076       逝去日子
18      347722       曾是拥有
19      347274         遥望
20  1855074233       再见理想
21    22831967         农民
22      347600      谁伴我闯荡
23      347705       天若有情
24      346075       真的爱你
25      346524       不可一世
26      346073        喜欢你
27      347702       灰色轨迹
28      346083      Amani
29      346082       俾面派对
开始下载歌曲: 海阔天空
http://m804.music.126.net/20211119220621/f1947abd936b313dc277734bca8aabc6/jdyyaac/5359/0f09/520c/d3dd238d118c4102cd67ef557f87da5c.m4a?authSecret=0000017d386e2366187